 ![alt text](https://s1.hostingkartinok.com/uploads/images/2024/01/6dbca656aa5e398971347080531cca4c.jpg)


In [1]:
import pandas as pd
import numpy as np
import torch 
import torch.nn as  nn
from torch.utils.data import TensorDataset, DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.preprocessing import StandardScaler
import time
from tqdm.autonotebook import tqdm, trange
import matplotlib.pyplot as plt

device = 'cuda' if torch.cuda.is_available() else 'cpu'

companies = ['GAZP', 'SBER', 'LKOH', 'GMKN', 'MGNT',
       'TATN', 'NVTK', 'SNGS', 'PLZL', 'SNGSP',
       'PIKK', 'ROSN', 'SBERP', 'CHMF', 'NLMK',
       'IRAO', 'YNDX', 'ALRS', 'RUAL', 'MTSS',
       'MAGN', 'PHOR', 'RTKM', 'VTBR', 'TATNP',
       'TCSG', 'AGRO', 'OZON', 'AFLT', 'AFKS',
       'FEES', 'TRNFP', 'CBOM', 'VKCO', 'ENPG',
       'SGZH', 'FIVE', 'POLY', 'MOEX', 'GLTR',
       'HYDR', 'FIXP']

/tmp/ipykernel_31110/3897907851.py:9: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
path = '~/Рабочий стол/ДИПЛОМ НОУТБУКИ/DL forcasitng diplom/Данные Мосбиржа/Total_MOEX_with_time_df'
total_df = pd.read_csv(path)

total_df.tradetime = total_df.close
total_df.drop(['close', 'pr_close'], 1, inplace = True)
total_df.rename(columns={'tradetime':'close'}, inplace=True)

data = total_df[total_df.secid == 'SBER'].drop('secid', 1).set_index('tradedate')


/tmp/ipykernel_31110/224406935.py:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  total_df.drop(['close', 'pr_close'], 1, inplace = True)
/tmp/ipykernel_31110/224406935.py:8: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  data = total_df[total_df.secid == 'SBER'].drop('secid', 1).set_index('tradedate')


In [3]:
def all_data_scaler(train_data, val_data):
    
    train_data = train_data.to_numpy()
    val_data = val_data.to_numpy()
    
    for i in range(data.shape[1]):
        
        train_series = train_data[:,i].reshape(-1, 1)
        val_series = val_data[:,i].reshape(-1, 1)
        
        scaler = StandardScaler()
        train_series = scaler.fit_transform(train_series)
        val_series = scaler.transform(val_series)
        
        train_data[:, i] = train_series.flatten()
        val_data[:, i] = val_series.flatten()
    
    return train_data, val_data 

In [4]:
data_train = data[ : round(data.shape[0]*0.53) ]
data_val = data[ round(data.shape[0]*0.60) : ]

In [5]:
data_train, data_val = all_data_scaler(data_train, data_val)

In [6]:
def sample_creator(df, lookback, horizon):
    """
    Функция нарезает данные скользящим окном, 
    lookback=исторический промежуток
    horizon=горизонт прогнозирования
    !Внимание! target переменная должна стоять в первом столбце!
    """
    X = []
    Y = np.array([np.zeros(horizon) for i in range(lookback, df.shape[0]-horizon+1 )])
    
    for i in range(lookback, df.shape[0]-horizon+1):
        x = df[i-lookback : i, :]
        X.append(x)
        y = df[i:i+horizon, 0]
        Y[i-lookback] += y
        
        
    X = torch.Tensor(np.array(X))
    Y = torch.Tensor(Y)
    return X, Y

In [7]:
lookback = 40
horizon = 1

x_train, y_train = sample_creator(data_train, lookback, horizon)
x_val, y_val = sample_creator(data_val, lookback, horizon)

print(f'x_train: {x_train.shape}, y_train: {y_train.shape}\nx_val: {x_val.shape}, y_val: {y_val.shape}')

x_train: torch.Size([50361, 40, 58]), y_train: torch.Size([50361, 1])
x_val: torch.Size([37998, 40, 58]), y_val: torch.Size([37998, 1])


In [8]:
train_dataset = TensorDataset(x_train, y_train)
val_dataset  = TensorDataset(x_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=150, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=150, shuffle=False)

In [9]:
from PatchTST_models import PatchTST, winrate

In [10]:
print(PatchTST.__doc__)


    Входыне данные:
    1) Кол-во рядов т.е. фичей: (M)
    2) Длина ряда: (L)
    3) Длина патча: (P)
    4) Страйд: (S)
    5) Кол-во головок памяти: (num_heads)
    6) Внутреннаяя размерность FFN: (dim_feedforward)
    7) Кол-во слоёв в энкодер блоке: (num_layers)
    8) Размерность энкодера: (d_model)
    9) Dropout: (dropout)
    
    
    


In [19]:
model = PatchTST(M=58,L=40,P=20, S=6, num_heads=4, dim_feedforward=1000, 
                 num_layers=3, d_model=512, dropout=0.1).to(device)

In [21]:
##### Train the model
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=3e-3)
scheduler = ReduceLROnPlateau(optimizer, 'min', factor=0.2, patience=3, verbose=True)

epochs = 50
early_stop_count = 0
min_val_loss = float('inf')

train_loss_per_epoch = []
val_loss_per_epoch = []
train_winrate = []
val_winrate = []



pbar = trange(epochs, desc="Epoch:")
since = time.time()

for epoch in pbar:
    model.train()
    train_losses = 0
    train_win = 0
    for batch in tqdm(train_loader, leave=False, desc="Train iter:"):
        x_batch, y_batch = batch
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        outputs = model(x_batch)
        outputs = outputs[:,[0]]
        loss = criterion(outputs, y_batch)
        loss.backward()
        train_losses += loss.item()
        optimizer.step()
        train_loss = train_losses/len(train_loader)
        train_win += winrate(y_batch, outputs)
    train_loss_per_epoch.append(train_loss)    
    train_winrate.append(train_win / len(train_loader))
    
    # Validation
    model.eval()
    val_losses = 0
    val_win = 0
    with torch.no_grad():
        for batch in tqdm(val_loader, leave=False, desc="Val iter:"):
            x_batch, y_batch = batch
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            outputs = model(x_batch)
            outputs = outputs[:,[0]]
            loss = criterion(outputs, y_batch)
            val_losses += loss.item()
            val_win += winrate(y_batch, outputs)
    val_loss = val_losses/len(val_loader)
    val_loss_per_epoch.append(val_loss)
    val_winrate.append(val_win / len(val_loader))
    
    scheduler.step(val_loss)
    if val_loss < 0.00012:
        torch.save(model.state_dict(), f'ВЕСА/PatchTST(M F=TI) 1D|W={val_win / len(val_loader)}, VL={val_loss}' )
    if val_loss < min_val_loss:
        min_val_loss = val_loss
        early_stop_count = 0
    else:
        early_stop_count += 1

    if early_stop_count >= 15:
        print("Early stopping!")
        break
    print(f"Epoch {epoch + 1}","/",f"{epochs},Train Loss: {train_loss:.6f} Validation Loss: {val_loss:.6f}")
time_elapsed = time.time() - since
print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))    

Epoch::   0%|          | 0/50 [00:00<?, ?it/s]

Train iter::   0%|          | 0/336 [00:00<?, ?it/s]

Val iter::   0%|          | 0/254 [00:00<?, ?it/s]

Epoch 1 / 50,Train Loss: 0.464224 Validation Loss: 1.593317


Train iter::   0%|          | 0/336 [00:00<?, ?it/s]

...

Train iter::   0%|          | 0/336 [00:00<?, ?it/s]

Val iter::   0%|          | 0/254 [00:00<?, ?it/s]

Epoch 15 / 50,Train Loss: 0.289290 Validation Loss: 1.928483


Train iter::   0%|          | 0/336 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
# Evaluation
model.eval()
predictions1 = []
with torch.no_grad():
    for batch in val_loader:
        x_batch, y_batch = batch
        x_batch = x_batch.to(device)
        outputs = model(x_batch)
        predictions1.extend(outputs.squeeze().tolist())
predictions1 = np.array(predictions1)

In [ ]:
winrate(y_val, torch.Tensor(predictions1))

In [ ]:
plt.plot(y_val[:,0][:100])
plt.plot(torch.Tensor(predictions1)[:,0][:100])
plt.legend(['Real','Pred'])